# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 1-1
Предполагается, что функция `solve_bp_decision(weights, n_bins)` решает «распознавательный» (decision) вариант задачи bin packing. На вход ей подаётся список весов и число корзин. Функция возвращает булевский ответ на вопрос «можно ли заданные веса раскидать по не более чем `n_bins` контейнерам? Напишите содержимое функции `solve_bp_evaluation`, которая возвращает оптимальное число корзин (решает evaluation variant задачи bin packing) и затем содержимое функции `solve_bp_search`, которая возвращает список номеров корзин, которые при каком-нибудь оптимальном распределении присваиваются весам из списка `weights` (корзины нумеруются с единицы). Каждая из следующих функций должна вызывать предыдущую не более чем полиномиальное число раз.

In [1]:
import itertools
from typing import List

In [2]:
def solve_bp_decision(weights: List[float], n_bins: int) -> bool:
    # stupid brute force
    for p in itertools.product(range(n_bins), repeat=len(weights)):
        bins = [0.0] * n_bins
        for i, bin in enumerate(p):
            bins[bin] += weights[i]
        if all(bin <= 1.0 for bin in bins):
            return True
    return False

In [3]:
def solve_bp_evaluation(weights: List[float]) -> int:
    left, right = 0, len(weights)
    while right - left > 1:
        middle = (left + right) // 2
        if solve_bp_decision(weights, n_bins=middle):
            right = middle
        else:
            left = middle
    return right

In [4]:
def solve_bp_search(weights: List[float]) -> List[int]:
    answer = solve_bp_evaluation(weights)
    bins = [[i] for i in range(len(weights))]

    merged_weights = weights[:]

    too_bad = False

    while not too_bad:
        too_bad = True
        for i, j in itertools.combinations(range(len(merged_weights)), 2):
            if merged_weights[i] + merged_weights[j] <= 1.0:
                too_bad = False
                new_weights = (merged_weights[:i] + merged_weights[i + 1:j] +
                               merged_weights[j + 1:] + [merged_weights[i] + merged_weights[j]])
                if solve_bp_evaluation(new_weights) == answer:
                    merged_weights = new_weights
                    bins = (bins[:i] + bins[i + 1:j] + bins[j + 1:] + [bins[i] + bins[j]])
                    break

    solution = [0] * len(weights)

    for idx, val in zip((i for bin in bins for i in bin),
                        (i + 1 for i, bin in enumerate(bins) for j in range(len(bin)))):
        solution[idx] = val

    return solution